# 基于Mindspore构建MinPooling层

MinPooling与MaxPooling相反，池化过程中采样最小值。由于Min Pooling使用场景极少,因此Mindspore中并未提供api,但是提供了maxpool,可以转换实现

# API使用方法参照MaxPooling

In [1]:
import numpy as np
import mindspore as ms
from mindspore import ops
ms.set_context(device_target="CPU")
data =   np.arange(9).reshape(1, 1, 3, 3).astype(np.float32)
print(data)
# minpool=- maxpool(-data)
op = ops.MaxPool(kernel_size=2, strides=2, pad_mode='same')
print(-op(   ms.Tensor(-data)   ))



[[[[0. 1. 2.]
   [3. 4. 5.]
   [6. 7. 8.]]]]
[[[[0. 2.]
   [6. 8.]]]]


# 自行实现Mindspore Min Pooling API

## 导入所需包

In [2]:
import mindspore.nn as nn
import numpy as np
import mindspore.common.dtype as mstype

from mindspore.ops import operations as P
from mindspore._checkparam import Rel, Validator as validator
from mindspore.common.tensor import Tensor
import mindspore.context as context

from mindspore.nn.cell import Cell

## 使用Mindspore官方定义的基类

In [3]:
class _PoolNd(Cell):
    """N-D  AvgPool"""

    def __init__(self, kernel_size, stride, pad_mode, data_format="NCHW"):
        """Initialize _PoolNd."""
        super(_PoolNd, self).__init__()
        validator.check_value_type('pad_mode', pad_mode, [str], self.cls_name)
        self.pad_mode = validator.check_string(pad_mode.upper(), ['VALID', 'SAME'], 'pad_mode', self.cls_name)
        self.format = validator.check_string(data_format, ['NCHW', 'NHWC'], 'format', self.cls_name)
        if context.get_context("device_target") != "GPU" and self.format == "NHWC":
            raise ValueError(f"For '{self.cls_name}, the 'NHWC' format only support in GPU target, but got device "
                             f"target {context.get_context('device_target')}.")

        def _check_int_or_tuple(arg_name, arg_value):
            validator.check_value_type(arg_name, arg_value, [int, tuple], self.cls_name)
            error_msg = f"For '{self.cls_name}', the '{arg_name}' must be an positive int number or " \
                        f"a tuple of two positive int numbers, but got {arg_value}"
            if isinstance(arg_value, int):
                if arg_value <= 0:
                    raise ValueError(error_msg)
            elif len(arg_value) == 2:
                for item in arg_value:
                    if isinstance(item, int) and item > 0:
                        continue
                    raise ValueError(error_msg)
            else:
                raise ValueError(error_msg)
            return arg_value

        self.kernel_size = _check_int_or_tuple('kernel_size', kernel_size)
        self.stride = _check_int_or_tuple('stride', stride)

    def construct(self, *inputs):
        pass

    def extend_repr(self):
        return 'kernel_size={kernel_size}, stride={stride}, pad_mode={pad_mode}'.format(**self.__dict__)

## 实现二维MinPool

In [4]:
class MinPool2d(_PoolNd):
    def __init__(self, kernel_size=1, stride=1, pad_mode="valid", data_format="NCHW"):
        """Initialize MaxPool2d."""
        super(MinPool2d, self).__init__(kernel_size, stride, pad_mode, data_format)
        self.max_pool = P.MaxPool(kernel_size=self.kernel_size,
                                  strides=self.stride,
                                  pad_mode=self.pad_mode,
                                  data_format=self.format)



    def construct(self, x):
        out = -self.max_pool(-x)
        return out



pool = MinPool2d(kernel_size=3, stride=1)
pool_max=nn.MaxPool2d(kernel_size=3, stride=1)
x = Tensor(np.random.randint(0, 10, [1, 2, 4, 4]), mstype.float32)
print(x)
output = pool(x)
print(output)
print(pool_max(x))

[[[[0. 2. 7. 3.]
   [9. 2. 8. 2.]
   [1. 1. 0. 7.]
   [8. 1. 3. 1.]]

  [[2. 0. 9. 7.]
   [3. 5. 6. 4.]
   [5. 7. 4. 9.]
   [9. 7. 9. 6.]]]]
[[[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [3. 4.]]]]
[[[[9. 8.]
   [9. 8.]]

  [[9. 9.]
   [9. 9.]]]]
